In [2]:
# Import der notwendigen Bibliotheken
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import scipy.optimize as sco
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

import yfinance as yf

# 1. Portfolios mit min. Varianz, min. VaR und max. Sharpe Ratio (1997-2001)

Wählen Sie 15 Unternehmen aus, die bereits seit 1997 an der Börse gelistet sind. Bestimmen Sie für Ihre 15 Aktien die Portfolios mit der **minimalen Varianz**, dem **minimalen VaR** und der **maximalen Sharpe Ratio**. Nutzen Sie dafür Tagesdaten Ihrer 15 Aktien von 1997-2001.

In [3]:
# Preise der 15 Aktien von 1997/01/01 - 2021/12/31 laden
yf.pdr_override()
tickers = ['MMM', 'TSLA', 'CSCO', 'V', 'GE', 'AMZN']
start = '2018-01-01'
end = '2021-12-31'

Adj_Close = web.get_data_yahoo(tickers, start, end)['Adj Close']
Adj_Close = pd.DataFrame(Adj_Close)
Adj_Close#.isna().sum()


[*********************100%***********************]  6 of 6 completed


,AMZN,CSCO,GE,MMM,TSLA,V
Date,,,,,,
2018-01-02,59.450500,33.135643,103.273819,199.068924,21.368668,110.811310
2018-01-03,60.209999,33.399994,104.250275,199.060471,21.150000,111.914467
2018-01-04,60.479500,33.494480,106.432907,201.662491,20.974667,112.330589
2018-01-05,61.457001,33.958370,106.490349,203.233810,21.105333,115.020782
2018-01-08,62.343498,34.310577,104.996964,202.574860,22.427334,115.485275
...,...,...,...,...,...,...
2021-12-23,171.068497,59.905964,73.065536,167.852905,355.666656,214.993698
2021-12-27,169.669495,61.002510,73.547447,169.512527,364.646667,215.996109
2021-12-28,170.660995,61.108318,74.076012,170.414291,362.823334,216.412964


In [22]:
# Berechnung der Logrenditen (weil additiv)
returns = pd.DataFrame(np.diff(np.log(Adj_Close).T).T)
returns.index = Adj_Close.index[1:]
returns.columns = tickers
returns

#Alternativ
#returns = pd.DataFrame(np.diff(np.log(Adj_Close.values).T).T)
#returns = np.log(Adj_Close/Adj_Close.shift(1))
#returns = np.log1p(Adj_Close.pct_change())

,MMM,TSLA,CSCO,V,GE,AMZN
Date,,,,,,
2018-01-03,0.012694,0.007946,0.009410,-0.000042,-0.010286,0.009906
2018-01-04,0.004466,0.002825,0.020720,0.012987,-0.008325,0.003711
2018-01-05,0.016033,0.013755,0.000539,0.007762,0.006210,0.023667
2018-01-08,0.014322,0.010318,-0.014123,-0.003247,0.060755,0.004030
2018-01-09,0.004665,-0.006279,0.015201,0.006194,-0.008118,-0.001929
...,...,...,...,...,...,...
2021-12-23,0.000184,0.012116,0.009943,0.013406,0.056020,-0.006167
2021-12-27,-0.008212,0.018139,0.006574,0.009839,0.024935,0.004652
2021-12-28,0.005827,0.001733,0.007161,0.005306,-0.005013,0.001928


In [23]:
#DataFrame auf den Zeitraum des Kalibrationsdatensatzes von 1997/01/01 - 2001/12/31 begrenzen
insample = returns
insample

#Alternativ
#insample = returns.iloc[0:1256]
#insample

,MMM,TSLA,CSCO,V,GE,AMZN
Date,,,,,,
2018-01-03,0.012694,0.007946,0.009410,-0.000042,-0.010286,0.009906
2018-01-04,0.004466,0.002825,0.020720,0.012987,-0.008325,0.003711
2018-01-05,0.016033,0.013755,0.000539,0.007762,0.006210,0.023667
2018-01-08,0.014322,0.010318,-0.014123,-0.003247,0.060755,0.004030
2018-01-09,0.004665,-0.006279,0.015201,0.006194,-0.008118,-0.001929
...,...,...,...,...,...,...
2021-12-23,0.000184,0.012116,0.009943,0.013406,0.056020,-0.006167
2021-12-27,-0.008212,0.018139,0.006574,0.009839,0.024935,0.004652
2021-12-28,0.005827,0.001733,0.007161,0.005306,-0.005013,0.001928


# 7 Zusatzaufaben

**Shortselling**

In [24]:
# Berechnung der Varianz-Covarianz-Matrix, mittleren Renditen und Festlegung von Alpha-Quantil und Zeithorizont
insample_cov = insample.cov() #Wiederholung
insample_mean_returns = insample.mean()




In [25]:
# Funktion 1
def calc_portfolio_std(weights, cov):
    portfolio_std = np.sqrt(np.dot(np.dot(weights.T, cov), weights)) * np.sqrt(252) #h=15x1 Matrix-> [h.transpose * cov] * h = [1x15 * 15x15] * 15x1 = 1x15 * 15x1 = 1x1
    return portfolio_std

# Funktion 2
def min_variance(cov):
    num_assets = len(cov)
    args = cov
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) #Nebenbedingung; ; eq für equality -> Gleichung != 0; np.sum(weights) - 1 = 0 (Summe der Gewichte müssen 1 ergeben und Gleichung muss gleich Null sein!)
    bound = (-1.0, 1.0)
    bounds = tuple(bound for assets in range(num_assets))
    result = sco.minimize(calc_portfolio_std, num_assets*[1/num_assets], args=args, 
                         method='SLSQP', bounds=bounds, constraints=constraints)
    return result

# fun: gibt den Zielfunktionswert (Min Varinaz) an & x: gibt die Gewichtungen der einzelnen Aktien im Portfolio an
min_port_variance = min_variance(insample_cov)
#min_port_variance

In [26]:
#Ergebnisse der Optimierung in DataFrame abspeichern
MIN_Variance = pd.DataFrame([round(x,2) for x in min_port_variance['x']], index=tickers).T #Gewichte auf 2 Nachkommastellen runden
MIN_Variance['Variance'] = min_port_variance['fun'] #Spalte für Zielfunktionswert
MIN_Variance

,MMM,TSLA,CSCO,V,GE,AMZN,Variance
0,0.29,0.11,0.01,0.42,-0.0,0.17,0.22914


In [27]:
#m

In [28]:
# Berechnung der Varianz-Covarianz-Matrix, mittleren Renditen und Festlegung von Alpha-Quantil und Zeithorizont
insample_cov = insample.cov() #Wiederholung
insample_mean_returns = insample.mean()
alpha = 0.05
days = 252

In [29]:
# Funktion 1
def calc_portfolio_VaR(weights, mean_returns, cov, alpha, days):
    portfolio_return = np.sum(weights * mean_returns) * days
    portfolio_std = np.sqrt(np.dot(np.dot(weights.T, cov), weights)) * np.sqrt(days)
    portfolio_VaR = abs(portfolio_return - (portfolio_std * stats.norm.ppf(1 - alpha))) #Erklären
    return portfolio_VaR

# Funktion 2
def min_VaR(mean_returns, cov, alpha, days):
    num_assets = len(mean_returns)
    args = (mean_returns, cov, alpha ,days)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (-1.0, 1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(calc_portfolio_VaR, num_assets*[1/num_assets], args=args, 
                         method='SLSQP', bounds=bounds, constraints=constraints)
    return result

# fun: gibt den Zielfunktionswert (Min Varinaz) an & x: gibt die Gewichtungen der einzelnen Aktien im Portfolio an
min_port_VaR = min_VaR(insample_mean_returns, insample_cov, alpha, days)
#min_port_VaR

In [30]:
#Ergebnisse der Optimierung in DataFrame abspeichern
MIN_VaR = pd.DataFrame([round(x,2) for x in min_port_VaR['x']], index=tickers).T
MIN_VaR['Value at Risk'] = min_port_VaR['fun']
MIN_VaR

,MMM,TSLA,CSCO,V,GE,AMZN,Value at Risk
0,0.46,0.44,-0.24,-0.34,0.33,0.36,0.123019


In [31]:
# Berechnung der Varianz-Covarianz-Matrix, mittleren Renditen und Festlegung des risikolosen Zinses
insample_cov = insample.cov() #Wiederholung
insample_mean_returns = insample.mean() #Wiederholung
rf = 0 #Begründung

In [32]:
# Funktion 1
def calc_neg_sharpe(weights, mean_returns, cov, rf):
    portfolio_return = np.sum(weights * mean_returns) * 252 #nur bei Logrenditen Addition möglich
    portfolio_std = np.sqrt(np.dot(np.dot(weights.T, cov), weights)) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return -sharpe_ratio

# Funktion 2
def max_sharpe_ratio(mean_returns, cov, rf):
    num_assets = len(mean_returns)
    args = (mean_returns, cov, rf)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) -1}) #Nebenbedingung
    bound = (-1.0, 1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(calc_neg_sharpe, num_assets*[1/num_assets], args=args, 
                          method='SLSQP', bounds=bounds, constraints=constraints)
    return result

# fun: gibt den Zielfunktionswert (Max Sharpe Ratio) an & x: gibt die Gewichtungen der einzelnen Aktien im Portfolio an
optimal_port_sharpe = max_sharpe_ratio(insample_mean_returns, insample_cov, rf)
#optimal_port_sharpe

In [33]:
#Ergebnisse der Optimierung in DataFrame abspeichern
MAX_SharpeRatio = pd.DataFrame([round(x,2) for x in optimal_port_sharpe['x']], index=tickers).T
MAX_SharpeRatio['Sharpe Ratio'] = - optimal_port_sharpe['fun']
MAX_SharpeRatio

,MMM,TSLA,CSCO,V,GE,AMZN,Sharpe Ratio
0,0.61,0.73,-0.5,-1.0,0.65,0.51,1.392199


In [34]:
# Erstellen eines DataFrames mit den Gewichtungen der drei Portfolios
insample_weights = pd.concat([x.iloc[:,:6] for x in [MIN_Variance, MIN_VaR, MAX_SharpeRatio]], axis=0)
insample_weights['Result'] = np.array([MIN_Variance.iloc[:,5], MIN_VaR.iloc[:,5], MAX_SharpeRatio.iloc[:,6]])
insample_weights.index = ['Min Variance', 'Min VaR', 'Max Sharpe Ratio']
insample_weights

,MMM,TSLA,CSCO,V,GE,AMZN,Result
Min Variance,0.29,0.11,0.01,0.42,-0.00,0.17,0.170000
Min VaR,0.46,0.44,-0.24,-0.34,0.33,0.36,0.360000
Max Sharpe Ratio,0.61,0.73,-0.50,-1.00,0.65,0.51,1.392199


In [36]:
insample_weights = insample_weights.drop(['Result'], axis=1)
insample_weights

,MMM,TSLA,CSCO,V,GE,AMZN
Min Variance,0.29,0.11,0.01,0.42,-0.00,0.17
Min VaR,0.46,0.44,-0.24,-0.34,0.33,0.36
Max Sharpe Ratio,0.61,0.73,-0.50,-1.00,0.65,0.51
